In [1]:
from QuantLib import *

In [2]:
# calendar data
today = Date(1,1,2016)
calendar = UnitedStates()
day_count = ActualActual()

# option data
ref_date = Date(1,1,2016)
tenor = Period(1, Years)
maturity_date = calendar.advance(ref_date, tenor)

spot_price = 1000
strike_price = 1000
option_type = Option.Call
sched = Schedule(ref_date, maturity_date, Period(Monthly),
                 calendar, Following, Following, DateGeneration.Forward, False)

sched_dates = [dt for dt in list(sched) if dt>=today]
times = [day_count.yearFraction(today, dt) for dt in sched_dates]
times2 = [calendar.businessDaysBetween(today, dt) for dt in sched_dates]

#for date, time in zip(sched_dates, times):
#    print("Date: {dt}, Time {t}".format(dt=date, t=time))
    
for date, time, time2 in zip(sched_dates, times, times2):
    print("Date: {dt}, Time {t}, Time2 {t2}".format(dt=date, t=time, t2=time2))
    
tg = TimeGrid(times)

# market
risk_free_rate = 0.001
dividend_rate =  0.0163
volatility = .2

calculation_date = today
Settings.instance().evaluationDate = calculation_date


Date: January 4th, 2016, Time 0.00819672131147541, Time2 0
Date: February 1st, 2016, Time 0.08469945355191257, Time2 19
Date: March 1st, 2016, Time 0.16393442622950818, Time2 39
Date: April 1st, 2016, Time 0.24863387978142076, Time2 62
Date: May 2nd, 2016, Time 0.3333333333333333, Time2 83
Date: June 1st, 2016, Time 0.41530054644808745, Time2 104
Date: July 1st, 2016, Time 0.4972677595628415, Time2 126
Date: August 1st, 2016, Time 0.5819672131147541, Time2 146
Date: September 1st, 2016, Time 0.6666666666666666, Time2 169
Date: October 3rd, 2016, Time 0.7540983606557377, Time2 190
Date: November 1st, 2016, Time 0.8333333333333334, Time2 210
Date: December 1st, 2016, Time 0.9153005464480874, Time2 230
Date: January 3rd, 2017, Time 1.0054794520547945, Time2 251


In [3]:
# Heston process
v0 = volatility*volatility  # spot variance
kappa = 0.1
theta = v0
sigma = 0.1
rho = -0.75

spot_handle = QuoteHandle(
    SimpleQuote(spot_price)
)
flat_ts = YieldTermStructureHandle(
    FlatForward(calculation_date, risk_free_rate, day_count)
)
dividend_yield = YieldTermStructureHandle(
    FlatForward(calculation_date, dividend_rate, day_count)
)
heston_process = HestonProcess(flat_ts,
                                dividend_yield,
                                spot_handle,
                                v0,
                                kappa,
                                theta,
                                sigma,
                                rho)

In [4]:
rng = GaussianRandomSequenceGenerator(
            UniformRandomSequenceGenerator(2 * (len(times)), UniformRandomGenerator()))
mpg = GaussianMultiPathGenerator(heston_process, times, rng, False)

In [5]:
sample_path = mpg.next()
multipath = sample_path.value()
for i in range(len(multipath)):
    print ("Time: {time}, Value: {value}, Vol: {vol}"
           .format(time=multipath[0].time(i), value=multipath[0].value(i), vol=multipath[1].value(i)))

Time: 0.0, Value: 1000.0, Vol: 0.04000000000000001
Time: 0.00819672131147541, Value: 1012.651253435154, Vol: 0.03908243823201241
Time: 0.08469945355191257, Value: 992.7878981279512, Vol: 0.032374700149109086
Time: 0.16393442622950818, Value: 965.4411386324166, Vol: 0.036443691316563616
Time: 0.24863387978142076, Value: 866.5078131991047, Vol: 0.04108791753421765
Time: 0.3333333333333333, Value: 931.1386484660869, Vol: 0.031712393374692976
Time: 0.41530054644808745, Value: 927.7904889051196, Vol: 0.03776050352151776
Time: 0.4972677595628415, Value: 964.0050649991236, Vol: 0.04001292979368612
Time: 0.5819672131147541, Value: 904.9314009396195, Vol: 0.045851746164350546
Time: 0.6666666666666666, Value: 856.9798923621412, Vol: 0.05293534515340567
Time: 0.7540983606557377, Value: 814.1801852590156, Vol: 0.05566736027068349
Time: 0.8333333333333334, Value: 836.8935363868496, Vol: 0.058467132958148114
Time: 0.9153005464480874, Value: 838.4022624117533, Vol: 0.06032124671010168
Time: 1.0054794

In [6]:
def generate_paths(numpaths, timesteps):
    paths = [0]*numpaths
    for i in range(numpaths):
        sample_path = mpg.next()
        multipath = sample_path.value()
        time = [multipath[0].time(j) for j in range(len(multipath))]
        value = [multipath[0].value(j) for j in range(len(multipath))]
        vol = [multipath[1].value(j) for j in range(len(multipath))]
        paths[i] = list(zip(time, value, vol))
    return paths

In [7]:
path = generate_paths(5, len(times))

In [8]:
path

[[(0.0, 1000.0, 0.04000000000000001),
  (0.00819672131147541, 977.8733792413718, 0.04137335739579938),
  (0.08469945355191257, 1033.2290236361453, 0.0349698799818572),
  (0.16393442622950818, 1067.3085052193323, 0.037101166917859765),
  (0.24863387978142076, 1117.6473173728152, 0.03414688112618449),
  (0.3333333333333333, 1192.5392996186608, 0.033506437795274326),
  (0.41530054644808745, 1288.817672422208, 0.03127035311056315),
  (0.4972677595628415, 1324.6765865198422, 0.03750962036633649),
  (0.5819672131147541, 1292.1332693035217, 0.036497153070384765),
  (0.6666666666666666, 1241.2067101501368, 0.034298141463567765),
  (0.7540983606557377, 1311.7058631375714, 0.041092442476185785),
  (0.8333333333333334, 1249.9567162215978, 0.04423461878051079),
  (0.9153005464480874, 1232.8266848818682, 0.047582593468920784),
  (1.0054794520547945, 1214.2146251146257, 0.05127992310037152)],
 [(0.0, 1000.0, 0.04000000000000001),
  (0.00819672131147541, 995.0610681474507, 0.041255858220875324),
  (0